In [1]:
%reload_ext autoreload 
%autoreload 2

import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import random 
import time
from utils import Dataset,ProgressBar
from tflearn.data_flow import DataFlow,DataFlowStatus,FeedDictFlow
from tflearn.data_utils import Preloader,ImagePreloader
import scipy
import pandas as pd
import xmltodict
import common
import tflearn
import copy
from cchess import *
from game_convert import convert_game

# a network predict select and move of Chinese chess, with minimal preprocessing

In [2]:
GPU_CORE = 0
BATCH_SIZE = 128
BEGINING_LR = 0.01
#TESTIMG_WIDTH = 500
model_name = '11_10_resnet20_shallow5'
data_dir = 'data/imsa-cbf/'

In [3]:
labels = common.board.create_uci_labels()
label2ind = dict(zip(labels,list(range(len(labels)))))

In [4]:
class ElePreloader(object):
    def __init__(self,datafile,batch_size=64):
        self.batch_size=batch_size
        content = pd.read_csv(datafile,header=None,index_col=None)
        self.filelist = [i[0] for i in content.get_values()]
        self.pos = 0
        self.feature_list = {"red":['A', 'B', 'C', 'K', 'N', 'P', 'R']
                             ,"black":['a', 'b', 'c', 'k', 'n', 'p', 'r']}
        self.batch_size = batch_size
        self.batch_iter = self.__iter()
        assert(len(self.filelist) > batch_size)
        self.game_iterlist = [None for i in self.filelist]
    
    def __iter(self):
        retx1,rety1,retx2,rety2 = [],[],[],[]
        filelist = []
        while True:
            for i in range(self.batch_size):
                if self.game_iterlist[i] == None:
                    if len(filelist) == 0:
                        filelist = copy.copy(self.filelist)
                        random.shuffle(filelist)
                    self.game_iterlist[i] = convert_game(filelist.pop(),feature_list=self.feature_list)
                game_iter = self.game_iterlist[i]
                
                try:
                    x1,y1 = game_iter.__next__()
                    x1 = np.transpose(x1,[1,2,0])
                    x1 = np.expand_dims(x1,axis=0)
                    retx1.append(x1)
                    #rety1.append(y1)
                    oney = np.zeros(len(labels))
                    oney[label2ind[''.join(y1)]] = 1
                    rety1.append(oney)

                    if len(retx1) >= self.batch_size:
                        yield (np.concatenate(retx1,axis=0),np.asarray(rety1))
                        retx1,rety1 = [],[]
                except :
                    self.game_iterlist[i] = None

    def __getitem__(self, id):
        
        x1,y1 = self.batch_iter.__next__()
        return x1,y1
        
    def __len__(self):
        return 10000

In [5]:
trainset = ElePreloader(datafile='data/train_list.csv',batch_size=BATCH_SIZE)
with tf.device("/gpu:{}".format(GPU_CORE)):
    coord = tf.train.Coordinator()
    trainflow = FeedDictFlow({
            'data':trainset,
        },coord,batch_size=BATCH_SIZE,shuffle=True,continuous=True,num_threads=1)
trainflow.start()

In [6]:
testset = ElePreloader(datafile='data/test_list.csv',batch_size=BATCH_SIZE)
with tf.device("/gpu:{}".format(GPU_CORE)):
    coord = tf.train.Coordinator()
    testflow = FeedDictFlow({
            'data':testset,
        },coord,batch_size=BATCH_SIZE,shuffle=True,continuous=True,num_threads=1)
testflow.start()

In [7]:
sample_x1,sample_y1 = trainflow.next()['data']

In [8]:
sample_x1,sample_y1 = testflow.next()['data']

In [9]:
sample_x1.shape,sample_y1.shape

((128, 10, 9, 14), (128, 2086))

In [10]:
labels[np.argmax(sample_y1[0])]

'c3c4'

In [11]:
np.sum(sample_x1[0],axis=-1)

array([[1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=uint64)

In [12]:
len(labels)

2086

In [13]:
np.sum(sample_x1[0],axis=-1).shape

(10, 9)

In [14]:
def res_block(inputx,name,training,block_num=2,filters=256,kernel_size=(3,3)):
    net = inputx
    for i in range(block_num):
        net = tf.layers.conv2d(net,filters=filters,kernel_size=kernel_size,activation=None,name="{}_res_conv{}".format(name,i),padding='same')
        net = tf.layers.batch_normalization(net,training=training,name="{}_res_bn{}".format(name,i))
        if i == block_num - 1:
            net = net + inputx #= tf.concat((inputx,net),axis=-1)
        net = tf.nn.elu(net,name="{}_res_elu{}".format(name,i))
    return net

def conv_block(inputx,name,training,block_num=1,filters=2,kernel_size=(1,1)):
    net = inputx
    for i in range(block_num):
        net = tf.layers.conv2d(net,filters=filters,kernel_size=kernel_size,activation=None,name="{}_convblock_conv{}".format(name,i),padding='same')
        net = tf.layers.batch_normalization(net,training=training,name="{}_convblock_bn{}".format(name,i))
        net = tf.nn.elu(net,name="{}_convblock_elu{}".format(name,i))
    # net [None,10,9,2]
    netshape = net.get_shape().as_list()
    print("inside conv block {}".format(str(netshape)))
    net = tf.reshape(net,shape=(-1,netshape[1] * netshape[2] * netshape[3]))
    net = tf.layers.dense(net,10 * 9,name="{}_dense".format(name))
    net = tf.nn.elu(net,name="{}_elu".format(name))
    return net

def res_net_board(inputx,name,training,filters=256):
    net = inputx
    net = tf.layers.conv2d(net,filters=filters,kernel_size=(3,3),activation=None,name="{}_res_convb".format(name),padding='same')
    net = tf.layers.batch_normalization(net,training=training,name="{}_res_bnb".format(name))
    net = tf.nn.elu(net,name="{}_res_elub".format(name))
    for i in range(NUM_RES_LAYERS):
        net = res_block(net,name="{}_layer_{}".format(name,i + 1),training=training)
        print(net.get_shape().as_list())
    print("inside res net {}".format(str(net.get_shape().as_list())))
    #net_unsoftmax = conv_block(net,name="{}_conv".format(name),training=training)
    return net

def get_scatter(name):
    with tf.variable_scope("Test"):
        ph = tf.placeholder(tf.float32,name=name)
        op = tf.summary.scalar(name,ph)
    return ph,op

In [16]:
?tf.layers.conv2d

In [15]:
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)

NUM_RES_LAYERS = 5

with tf.device("/gpu:{}".format(GPU_CORE)):
    X = tf.placeholder(tf.float32,[None,10,9,14])
    nextmove = tf.placeholder(tf.float32,[None,len(labels)])
    score = tf.placeholder(tf.float32,[None,1])
    
    training = tf.placeholder(tf.bool,name='training_mode')
    learning_rate = tf.placeholder(tf.float32)
    global_step = tf.train.get_or_create_global_step()
    
    body = res_net_board(X,"selectnet",training=training)
    with tf.variable_scope("policy_head"):
        policy_head = tf.layers.conv2d(body, 2, 1, padding='SAME')
        policy_head = tf.contrib.layers.batch_norm(policy_head, center=False, epsilon=1e-5, fused=True,
                                                    is_training=training, activation_fn=tf.nn.relu)

        # print(self.policy_head.shape)  # (?, 9, 10, 2)
        policy_head = tf.reshape(policy_head, [-1, 9 * 10 * 2])
        policy_head = tf.contrib.layers.fully_connected(policy_head, len(labels), activation_fn=None)
        #self.policy_head.append(policy_head)    # 保存多个gpu的策略头结果（走子概率向量）

    # 价值头
    with tf.variable_scope("value_head"):
        value_head = tf.layers.conv2d(body, 1, 1, padding='SAME')
        value_head = tf.contrib.layers.batch_norm(value_head, center=False, epsilon=1e-5, fused=True,
                                        is_training=training, activation_fn=tf.nn.relu)
        # print(self.value_head.shape)  # (?, 9, 10, 1)
        value_head = tf.reshape(value_head, [-1, 9 * 10 * 1])
        value_head = tf.contrib.layers.fully_connected(value_head, 256, activation_fn=tf.nn.relu)
        value_head = tf.contrib.layers.fully_connected(value_head, 1, activation_fn=tf.nn.tanh)
    
    net_unsoftmax = policy_head
    

    with tf.variable_scope("Loss"):
        policy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=nextmove,logits=net_unsoftmax))
        #loss_summary = tf.summary.scalar("move_loss",policy_loss)
        value_loss = tf.losses.mean_squared_error(labels=score, predictions=value_head) 
        regularizer = tf.contrib.layers.l2_regularizer(scale=0.0001)
        regular_variables = tf.trainable_variables()
        l2_loss = tf.contrib.layers.apply_regularization(regularizer, regular_variables)
        multitarget_loss = value_loss + policy_loss + l2_loss
        
    net_softmax = tf.nn.softmax(net_unsoftmax)

    correct_prediction = tf.equal(tf.argmax(nextmove,1), tf.argmax(net_softmax,1))
    
    with tf.variable_scope("Accuracy"):
        accuracy_select = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
        train_op = optimizer.minimize(policy_loss,global_step=global_step)
        

[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
[None, 10, 9, 256]
inside res net [None, 10, 9, 256]


In [16]:
sess.run(tf.global_variables_initializer())
tf.train.global_step(sess, global_step)

0

In [17]:
!mkdir models

mkdir: 无法创建目录"models": 文件已存在


In [18]:
import os
if not os.path.exists("models/{}".format(model_name)):
    os.mkdir("models/{}".format(model_name))

In [ ]:
N_BATCH = 10000
N_BATCH_TEST = 300

In [ ]:
restore = False
N_EPOCH = 100
DECAY_EPOCH = 20

class ExpVal:
    def __init__(self,exp_a=0.97):
        self.val = None
        self.exp_a = exp_a
    def update(self,newval):
        if self.val == None:
            self.val = newval
        else:
            self.val = self.exp_a * self.val + (1 - self.exp_a) * newval
    def getval(self):
        return round(self.val,2)
    
expacc_move = ExpVal()
exploss = ExpVal()


begining_learning_rate = 1e-1

pred_image = None
if restore == False:
    train_epoch = 1
    train_batch = 0
for one_epoch in range(train_epoch,N_EPOCH):
    train_epoch = one_epoch
    pb = ProgressBar(worksum=N_BATCH * BATCH_SIZE,info=" epoch {} batch {}".format(train_epoch,train_batch))
    pb.startjob()
    
    for one_batch in range(N_BATCH):
        if restore == True and one_batch < train_batch:
            pb.auto_display = False
            pb.complete(BATCH_SIZE)
            pb.auto_display = True
            continue
        else:
            restore = False
        train_batch = one_batch
        
        batch_x,batch_y = trainflow.next()['data']
        # learning rate decay strategy
        batch_lr = begining_learning_rate * 10 ** -(one_epoch // DECAY_EPOCH)
        
        _,step_loss,step_acc_move,step_value = sess.run(
            [train_op,policy_loss,accuracy_select,global_step],feed_dict={
                X:batch_x,nextmove:batch_y,learning_rate:batch_lr,training:True
            })
        
        step_acc_move *= 100
        
        expacc_move.update(step_acc_move)
        exploss.update(step_loss)

       
        pb.info = "EPOCH {} STEP {} LR {} ACC {} LOSS {} ".format(
            one_epoch,one_batch,batch_lr,expacc_move.getval(),exploss.getval())
        
        pb.complete(BATCH_SIZE)
    print()
    pb = ProgressBar(worksum=N_BATCH // 4 * BATCH_SIZE,info=" epoch {}".format(train_epoch))
    pb.startjob()
    losses = []
    accs = []
    for one_batch in range(N_BATCH // 4):
        batch_x,batch_y = testflow.next()['data']
        # learning rate decay strategy
        batch_lr = begining_learning_rate * 10 ** -(one_epoch // DECAY_EPOCH)
        
        step_loss,step_acc_move,step_value = sess.run(
            [policy_loss,accuracy_select,global_step],feed_dict={
                X:batch_x,nextmove:batch_y,learning_rate:batch_lr,training:False
            })
        
        step_acc_move *= 100
        losses.append(step_loss)
        accs.append(step_acc_move)
        pb.complete(BATCH_SIZE)
    print("EPOCH {} valid loss {} acc {}".format(train_epoch,np.average(losses),np.average(accs)))
    print()
    saver = tf.train.Saver(var_list=tf.global_variables())
    saver.save(sess,"models/{}/model_{}".format(model_name,one_epoch))

EPOCH 1 STEP 9999 LR 0.1 ACC 26.91 LOSS 2.87  100.00 % [==================================================>] 1280000/1280000 	 used:1447s eta:0 ss s
 epoch 1 100.00 % [==================================================>] 320000/320000 	 used:233s eta:0 sEPOCH 1 valid loss 2.881636381149292 acc 26.750625

EPOCH 2 STEP 9999 LR 0.1 ACC 30.91 LOSS 2.61  100.00 % [==================================================>] 1280000/1280000 	 used:1430s eta:0 ss
 epoch 2 100.00 % [==================================================>] 320000/320000 	 used:243s eta:0 sEPOCH 2 valid loss 2.6430976390838623 acc 30.1371875

EPOCH 3 STEP 9999 LR 0.1 ACC 33.84 LOSS 2.46  100.00 % [==================================================>] 1280000/1280000 	 used:1428s eta:0 ss
 epoch 3 100.00 % [==================================================>] 320000/320000 	 used:242s eta:0 sEPOCH 3 valid loss 2.526669502258301 acc 32.25875

EPOCH 4 STEP 9999 LR 0.1 ACC 33.25 LOSS 2.44  100.00 % [=============================